## Aquisição de dados do Google Trends
- O script foi desenvolvido baseando-se no download dos arquivos .csv disponilizados pelo próprio Google Trends;
- A escolha mencionada foi tomada com o intuito de facilitar a aquisição dos dados, evitando interações mais diretas com o HTML da página;
- Os arquivos .csv são baixados e posteriormente seus dados são interpretados em dataframes;
- Os dataframes são manipulados seguindo algumas especificações e posteriormente é feito o upload dos dados para o postgres;

In [415]:
#===Bibliotecas a serem utilizadas
from selenium.webdriver.chrome.options import Options
from sqlalchemy import create_engine
from selenium import webdriver
from datetime import datetime
from time import sleep
import psycopg2 as pg
import pandas as pd
import os as os
import sys

### *1.0 - Criação de diretórios e definição de destino padrão para download*

#### Criação de diretórios específicos com a biblioteca os
- Criando diretório 'trends_api' caso o mesmo não exista;
- O diretório 'trends_api' servirá como destino padrão dos downloads dos arquivos csv;

In [416]:
#===Criando pasta para armanezar os arquivos csv baixados
pasta = 'trends_api'
endereco = os.path.join(os.path.expanduser("~"), pasta)

if not os.path.exists(pasta):
    os.mkdir(pasta)
    
#===Definindo caminho para download dos arquivos csv
opcoes = Options()

prefs = {'download.default_directory': endereco}
opcoes.add_experimental_option('prefs', prefs)

### *2.0 - Download dos arquivos e criação de DataFrames provisórios*
#### Navegando na página com selenium e realizando downloads
- Caso o site seja acessado diretamente com (url + pesquisa) será automaticamente barrado, por isso a navegação passo a passo;
- A variável 'termo' se refere ao termo sobre o qual serão adquiridas as informações de tendências;
- As variáveis 'intervaloDe' e 'intervaloAte' se referem ao intervalo de tempo que os dados serão extraídos ('intervaloDe: data de início; intervaloAte: data de término);
- Os botões de pesquisa, filtragem e download são locaizados através de seus xpath's;
- Devido ao fato de que caso hajam falhas de conexão com a internet os campos xpath não vão ser encontrados, a estrutura de busca é feita para iterar até 10 vezes sobre a página em caso de erros por falha de conexão;
- É possível diminuir os tempos de sleep de acordo com a qualidade atual da conexão com a internet;
>IMPORTANTE: Os dados de data devem ser inseridos no formato MES/DIA/ANO e não existem dados disponíveis anteriores ao ano de 2004. Além disso, o código não funciona caso esteja em modo headless, uma vez que é necessário que carreguem alguns recursos gráficos para que funcione;

In [417]:
#====================Definindo parâmetros====================#
termo = 'tatu'
intervaloDe = '09/20/2021'
intervaloAte = '09/22/2023'


flag_concluido = False
for i in range(9):
    try:    
        #==========Navegando pela página
        url = 'https://trends.google.com.br'
        navegador = webdriver.Chrome(options=opcoes)
            
        navegador.get(url)
        sleep(2)
        navegador.find_element('xpath', '//*[@id="i9"]').click()
        sleep(2)
        navegador.find_element('xpath', '//*[@id="i9"]').send_keys(termo)
        sleep(2)
        navegador.find_element('xpath', '//*[@id="yDmH0d"]/c-wiz/div/div[2]/div[4]/div[1]/c-wiz[1]/div/div[1]/div[3]/div/div[2]/div[3]/div/button/span[6]').click()
        sleep(2)
        navegador.find_element('xpath', '//*[@id="select_value_label_9"]/span[1]/div').click()
        sleep(2)

        #==========Fechando possível cookie
        try:
            navegador.find_element('xpath','//*[@id="cookieBar"]/div/span[2]/a[2]').click()
            sleep(1)
        except:
            pass

        #==========Definindo intervalo de tempo
        navegador.find_element('xpath', '/html/body/div[8]/md-select-menu/md-content/md-option[10]/div').click()
        sleep(3)

        nums = ['1', '2']
        intervalos = [intervaloDe, intervaloAte]
        for i in range(len(nums)):
            navegador.find_element('xpath', f'/html/body/div[2]/div[4]/md-dialog/md-tabs/md-tabs-content-wrapper/md-tab-content[1]/div/md-content/form/div[{nums[i]}]/md-datepicker/div/input').clear()
            sleep(2)
            navegador.find_element('xpath', f'/html/body/div[2]/div[4]/md-dialog/md-tabs/md-tabs-content-wrapper/md-tab-content[1]/div/md-content/form/div[{nums[i]}]/md-datepicker/div/input').send_keys(intervalos[i])
            sleep(2)

        navegador.find_element('xpath', '/html/body/div[2]/div[4]/md-dialog/md-dialog-actions/button[2]').click()
        sleep(5)

        #==========Realizando downloads
        nums = ['1','2','3']
        for num in nums:
            buttom = f'/html/body/div[2]/div[2]/div/md-content/div/div/div[{num}]/trends-widget/ng-include/widget/div/div/div/widget-actions/div/button[1]/i'
            navegador.find_element('xpath', buttom).click()
            sleep(4)
            
        flag_concluido = True
        navegador.quit()
        break
        
    except:
        navegador.quit()
        pass
    
#==========Checagem se os arquivos foram baixados ou não
if flag_concluido == False:
    print('''ERRO:Os arquivos não foram baixados. 
    Prováveis causas: 
    1 - Identificação dos xpath's (mudança no html da página);
    2 - Erro de inserção dos dados de 'intervaloDe' e 'intervaloAte';
    3 - Conexão muito instável com a internet impossibilitou o carregamento dos xpath's;
    ''')
    sys.exit(1)

#### Criando DataFrames provisórios com os arquivos csv
- Criação de 3 dataframes referentes, respectivamente, aos dados de interesse por sub região, assuntos relacionados e interesse por data;
- Os parâmetros na criação do dataframe são passados a fim de corrirgir especificidades de cada csv cedido pelo Google Trends;
- Os dataframes provisórios tem a terminação 'PROV' a fim de facilitar sua identificação;

In [418]:
#===Criando dataframes
arquivos = ['geoMap.csv', 'relatedEntities.csv', 'multiTimeline.csv']

#===Interesse por sub região
try:
    df_subRegiaoPROV = pd.read_csv(f'{endereco}\\{arquivos[0]}', delimiter=',', skiprows=3, header=None, encoding='utf-8')
    colunas_sub = ['estado','interesse']
    df_subRegiaoPROV.columns = colunas_sub
except:
    df_subRegiaoPROV = pd.DataFrame(columns={'estado':[],'interesse':[]})
  
#===Assuntos relacionados
try:
    df_assuntosRelacPROV = pd.read_csv(f'{endereco}\\{arquivos[1]}', delimiter=',', skiprows=4, header=None, encoding='utf-8')
    colunas_assu = ['assunto', 'interesse']
    df_assuntosRelacPROV.columns = colunas_assu
except:
    df_assuntosRelacPROV = pd.DataFrame(columns={'assunto':[],'interesse':[]})

#===Interesse por data
try:
    df_interesseDataPROV = pd.read_csv(f'{endereco}\\{arquivos[2]}', skiprows=3, header=None, encoding='utf-8')
    colunas = ['data','interesse']
    df_interesseDataPROV.columns = colunas
except:
    df_interesseDataPROV = pd.DataFrame(columns={'data':[],'interesse':[]})

### *3.0 Tratamento dos DataFrames provisórios*
#### Tratando dataframe de assuntos relacionados ao termo e porcentagens de aumento
- O arquivo .csv possui dados os quais refletem o aumento da tendência de pesquisa em porcentagem, fugindo do padrão numérico de interesse;
- O dataframe foi filtrado deixando somente as linhas que seguem o padrão numérico de interesse (1 a 100), então valores menores que 1, porcentagens ou caracteres foram excluídos;
- Os dados que possuíam porcentagem (caso existam) foram armazenados no data frame 'df_aumentAssuntoPROV';

In [419]:
#===============Tratamento dos dados do dataframe de assuntos relacionados===============#
list_assuntosPorc = []
list_valoresPorc = []

list_assuntos = []
list_valores = []

#===Separação dos dados para criação do dataframe de porcentagens
nums = ['1','2','3','4','5','6','7','8','9','0']
for i in range(len(df_assuntosRelacPROV)):
    valor = df_assuntosRelacPROV['interesse'].iloc[i]
    assunto = df_assuntosRelacPROV['assunto'].iloc[i]
    valor2 = (str(valor))[-1]
    valor1 = (str(valor))[0]
    
    if valor1 in nums:
        list_valores.append(valor)
        list_assuntos.append(assunto)
    elif valor2 == '%':
        list_valoresPorc.append(valor)
        list_assuntosPorc.append(assunto)

#===Criação dos novos dataframes
df_aumentAssuntoPROV = pd.DataFrame({'assunto': list_assuntosPorc, 'aumento': list_valoresPorc})
df_assuntosRelacPROV = pd.DataFrame({'assunto' : list_assuntos, 'interesse' : list_valores})


#===============Tratamento do dataframe de porcentagens (retirando caracteres não numéricos)===============#
list_provisoria_valores = []
list_assunto = []
    
for i in range(len(df_aumentAssuntoPROV)):
    list_provisoria_valores.append(df_aumentAssuntoPROV['aumento'].iloc[i])
    list_assunto.append(df_aumentAssuntoPROV['assunto'].iloc[i])
    
#===Tratando as células e removendo caracteres que não sejam numéricos
lista_porcentagens = []
nums = '1234567890'
for i in range(len(list_provisoria_valores)):
    string = list(list_provisoria_valores[i])
    for i in range(len(string)):
        if string[i] not in nums:
            string[i] = ''
    string_final = ''.join(string)
    lista_porcentagens.append(string_final)

#===Substituindo valores já corrigidos dentro do dataframe de aumento de porcentagem
for i in range(len(df_aumentAssuntoPROV)):
    valor = df_aumentAssuntoPROV['aumento'].iloc[i]
    df_aumentAssuntoPROV['aumento'].iloc[i] = df_aumentAssuntoPROV['aumento'].iloc[i].replace(valor, lista_porcentagens[i])

#### Tratamento de dados numéricos inteiros nas tabelas de subregiao e interesse por data
- Alguns valores podem conter o sinal '<', portanto é necessário que os mesmos sejam tratados;
- A fim de simplificação, esses dados foram substituídos por números 0, o que significa que há pouco (menos que um ponto) ou nenhum interesse;

In [420]:
nums = ['1','2','3','4','5','6','7','8','9','0']

#===Redefinindo tipos de data para ser possível a atuação do .replace()
df_subRegiaoPROV['interesse'] = df_subRegiaoPROV['interesse'].astype(str)
df_interesseDataPROV['interesse'] = df_interesseDataPROV['interesse'].astype(str)

#===Replace dataframe de interesse por sub regiao
for i in range(len(df_subRegiaoPROV)):
    valor = df_subRegiaoPROV['interesse'].iloc[i]
    valor1 = (str(valor))[0]
    
    if valor1 not in nums:
        df_subRegiaoPROV['interesse'].iloc[i] = df_subRegiaoPROV['interesse'].iloc[i].replace(valor, '0')
 

#===Replace dataframe de interesse por assunto relacionado
for i in range(len(df_interesseDataPROV)):
    valor = df_interesseDataPROV['interesse'].iloc[i]
    valor1 = (str(valor))[0]
    
    if valor1 not in nums:
        df_interesseDataPROV['interesse'].iloc[i] = df_interesseDataPROV['interesse'].iloc[i].replace(valor, '0')

#### Ajustando dados não inteiros nas colunas de quantidade de interesse
- Algumas vezes o site google trends fornece dados com formato '2.0' (float), o que impede que o restante do código seja executado, isso está sendo corrigido aqui;

In [421]:
dataframes = [df_subRegiaoPROV, df_interesseDataPROV, df_assuntosRelacPROV]

#===Substituindo dados nan por '0' para evitar possíveis erros
for dataframe in dataframes:
    dataframe['interesse'] = dataframe['interesse'].fillna(0)

#===Transformando dados float em int
for dataframe in dataframes:
    for j in range(len(dataframe)):
        valor = dataframe['interesse'].iloc[j]
        valorRep = float(valor)
        valorRep = int(valorRep)
        valorRep = str(valorRep)
        dataframe['interesse'].iloc[j] = dataframe['interesse'].iloc[j].replace(valor, valorRep)

#===Excessão para o dataframe 'df_aumentAssuntoPROV' pois o mesmo não possui o mesmo nome de coluna que os demais 
df_aumentAssuntoPROV['aumento'] = df_aumentAssuntoPROV['aumento'].fillna(0)

for i in range(len(df_aumentAssuntoPROV)):
    valor = df_aumentAssuntoPROV['aumento'].iloc[i]
    valorRep = float(valor)
    valorRep = int(valorRep)
    valorRep = str(valorRep)
    df_aumentAssuntoPROV['aumento'].iloc[i] = df_aumentAssuntoPROV['aumento'].iloc[i].replace(valor, valorRep)

### *4.0 - Criação dos DataFrames  finais*

#### Criação dos dataframes finais para posterior upload
- A respeito do dataframe de interesse por UF, o mesmo possui as colunas de intervalo de tempo (data de início da pesquisa e data de fim da pesquisa), o termo buscado, a unidade federativa e a quantidade de interesse;
- Já o dataframe de interesse por data terá as colunas de intervalo de tempo (data de início da pesquisa e data de fim da pesquisa), data de registro, termo buscado e quantidade de interesse;
- A respeito do dataframe de interesse por assunto relacionado, o mesmo terá as colunas de intervalo de tempo (data de início da pesquisa e data de fim da pesquisa), o termo buscado, assunto relacionado e quantidade de interesse;
- Por fim, o dataframe de porcentage de aumento do interesse por assunto possuirá as colunas de intervalo de tempo (data de início e data de término), o termo pesquisado, assunto e a quantidade em porcentagem do aumento de interesse;
> Vale destacar que os dados de aumento de interesse em porcentagem se referem SOMENTE a pesquisas que tiveram um aumento REPENTINO dentro o intervalo estipulado;

In [422]:
#===============Criação do dataframe de interesse por UF 'df_subReg'===============#
colunas = ['Data_Inicio', 'Data_Fim', 'Termo_Busca', 'UF', 'Quant_Interesse']
df_subReg = pd.DataFrame(columns = colunas)

datainic = []
datafim = []
termobuscas = []
interesses = []
estados = []

for i in range(len(df_subRegiaoPROV)):
    estados.append(df_subRegiaoPROV['estado'].iloc[i])
    interesses.append(df_subRegiaoPROV['interesse'].iloc[i])
    datainic.append(intervaloDe)
    datafim.append(intervaloAte)
    termobuscas.append(termo)

lista_auxiliar = [datainic, datafim, termobuscas, estados, interesses]
for i in range(len(lista_auxiliar)):
    df_subReg[colunas[i]] = lista_auxiliar[i]
    
#===============Criação do dataframe de interesse por data 'df_dataInteresse'===============#
colunas = ['Intervalo_De','Intervalo_Ate','Data_Registro', 'Termo_Busca', 'Quant_Interesse']
df_dataInteresse = pd.DataFrame(columns = colunas)

datas = []
termobuscas = []
datainic = []
datafim = []
interesses = []

for i in range(len(df_interesseDataPROV)):
    datas.append(df_interesseDataPROV['data'].iloc[i])
    interesses.append(df_interesseDataPROV['interesse'].iloc[i])
    termobuscas.append(termo)
    datainic.append(intervaloDe)
    datafim.append(intervaloAte)

lista_auxiliar = [datainic, datafim, datas, termobuscas, interesses]
for i in range(len(lista_auxiliar)):  
    df_dataInteresse[colunas[i]] = lista_auxiliar[i]

    
#===============Criação do dataframe de interesse por assunto relacionado 'df_assuntoRelac'===============#
colunas = ['Data_Inicio', 'Data_Fim', 'Termo_Busca', 'Assunto_Relacionado', 'Quant_Interesse']
df_assuntoRelac = pd.DataFrame(columns = colunas)

datainic = []
datafim = []
termobuscas = []
interesses = []
assuntos = []

for i in range(len(df_assuntosRelacPROV)):
    assuntos.append(df_assuntosRelacPROV['assunto'].iloc[i])
    interesses.append(df_assuntosRelacPROV['interesse'].iloc[i])
    datainic.append(intervaloDe)
    datafim.append(intervaloAte)
    termobuscas.append(termo)

lista_auxiliar = [datainic, datafim, termobuscas, assuntos, interesses]
for i in range(len(lista_auxiliar)):  
      df_assuntoRelac[colunas[i]] = lista_auxiliar[i]

#===============Criação do dataframe de porcentagem de aumento por termo de pesquisa 'df_aumentoAssunto'====================#
colunas = ['Data_Inicio', 'Data_Fim', 'Termo_Busca', 'Assunto', 'Porcentagem_Aumento']
df_aumentoAssunto = pd.DataFrame(columns = colunas)
    
datainic = []
datafim = []
termobuscas = []
assunto = []
    
for i in range(len(df_aumentAssuntoPROV)):
    assunto.append(df_aumentAssuntoPROV['assunto'].iloc[i])
    datainic.append(intervaloDe)
    datafim.append(intervaloAte)
    termobuscas.append(termo)
    
lista_auxiliar = [datainic, datafim, termobuscas, assunto, lista_porcentagens]
for i in range(len(lista_auxiliar)):
    df_aumentoAssunto[colunas[i]] = lista_auxiliar[i] 

#### Redefinindo os tipos de dado de cada coluna
- Redefinição dos tipos de dados por culuna colocando os dados de quantidade de interesse como 'int', os dados que são texto como 'object' e as datas como 'datetime' (com excessão da coluna de data_registro, a mesma foi classificada como 'timestamp with time zone';
- Caso os valores de interesse por estado estejam vazios eles são preenchidos com 0 para posterior conversão dos dados;

In [423]:
#=====DataFrame df_aumentoAssunto=====#
colunas_aumentoAssunto = {
    'Porcentagem_Aumento': int,
    'Termo_Busca': object,
    'Assunto': object,
    'Data_Inicio': 'date',
    'Data_Fim': 'date'
}

#=====DataFrame df_subReg=====#
colunas_subReg = {
    'Quant_Interesse': int,
    'Termo_Busca': object,
    'UF': object,
    'Data_Inicio': 'date',
    'Data_Fim': 'date'
}

#=====DataFrame df_dataInteresse=====#
colunas_dataInteresse = {
    'Intervalo_De': 'date',
    'Intervalo_Ate': 'date',
    'Data_Registro': 'date',
    'Termo_Busca': object,
    'Quant_Interesse': int
}

#=====DataFrame df_assuntoRelac=====#
colunas_assuntoRelac = {
    'Quant_Interesse': int,
    'Data_Inicio': 'date',
    'Data_Fim': 'date',
    'Termo_Busca': object,
    'Assunto_Relacionado': object
}

#===============Convertendo os tipos de dados das colunas===============#
def conversor_colunas(dataframe, conversoes):
    for coluna, tipo_de_dado in conversoes.items():
        if tipo_de_dado == 'date':
            dataframe[coluna] = pd.to_datetime(dataframe[coluna])
        else:
            dataframe[coluna] = dataframe[coluna].astype(tipo_de_dado)

df_subReg = df_subReg.fillna(0)

conversor_colunas(df_aumentoAssunto, colunas_aumentoAssunto)
conversor_colunas(df_dataInteresse, colunas_dataInteresse)
conversor_colunas(df_assuntoRelac, colunas_assuntoRelac)
conversor_colunas(df_subReg, colunas_subReg)

### *5.0 - Upload dos dataframes para o banco de dados*

#### Upload dos dataframes para o banco de dados postgres
- Estabelecendo a conexão com o banco de dados postgres;
- Com a conexão estabelecida, são criadas as 4 tabelas de dados dentro do banco de dados tendo seus nomes formados por 'termo de pesquisa' + 'início do seu tipo de dado' + 'número da execução';
- Após a criação das tabelas, é feito o upload de cada dataframe para sua respectiva tabela;
- Depois de todo o processo feito, os arquivos csv originais são apagados a fim de evitar inconsistências nas próximas execuções;

In [424]:
#===Estabelecendo conexão com banco de dados
host = 'SEU HOST'
dbname = 'NOME DA BASE DE DADOS PARA ONDE OS DADOS VÃO'
user = 'SEU USUÁRIO'
password = 'SUA SENHA'
sslmode = ''
port = ''

conect_string = 'host={} user={} dbname={} password={} sslmode={}'.format(host, user, dbname, password, sslmode)
conec = pg.connect(conect_string)
cursor = conec.cursor()
print(f'Conexão com o banco de dados "{dbname}" estabelecia')

Conexão com o banco de dados "testegoogletrends" estabelecia


#### Criação de tabelas
- Caso ainda não existam, são criadas as tabelas 'aumento_assunto', 'sub_regiao', 'interesse_por_data' e 'assunto_relacionado' dentro do banco de dados;
- As tabelas armanezam, respectivamente, os dados de porcentagem de aumento do interesse por assunto, interesse por sub região, interesse geral por intervalo de tempo específico, e assunto relacionado;

In [425]:
#===Criando tabelas dentro do banco de dados
nomes_tabelas = ['aumento_assunto', 'sub_regiao', 'interesse_por_data', 'assunto_relacionado']

#===============Tabela Aumento_Assunto===============#
consulta_criar_tabela = f'''
CREATE TABLE IF NOT EXISTS {nomes_tabelas[0]} (
    Data_Inicio date,
    Data_Fim date,
    Termo_Busca varchar,
    Assunto varchar,
    Porcentagem_Aumento int
);
'''
cursor.execute(consulta_criar_tabela)
conec.commit()

#===============Tabela Interesse_Por_Data===============#
consulta_criar_tabela = f'''
CREATE TABLE IF NOT EXISTS {nomes_tabelas[2]} (
    Intervalo_De date,
    Intervalo_Ate date,
    Data_Registro timestamp with time zone,
    Termo_Busca varchar,
    Quant_Interesse int
);
'''
cursor.execute(consulta_criar_tabela)
conec.commit()

#===============Tabela Sub_Regiao===============#
consulta_criar_tabela = f'''
CREATE TABLE IF NOT EXISTS {nomes_tabelas[1]} (
    Data_Inicio date,
    Data_Fim date,
    Termo_Busca varchar,
    UF varchar,
    Quant_Interesse int
);
'''
cursor.execute(consulta_criar_tabela)
conec.commit()

#===============Tabela Assunto_Relacionado===============#
consulta_criar_tabela = f'''
CREATE TABLE IF NOT EXISTS {nomes_tabelas[3]} (
    Data_Inicio date,
    Data_Fim date,
    Termo_Busca varchar,
    Assunto_Relacionado varchar,
    Quant_Interesse int
);
'''
cursor.execute(consulta_criar_tabela)
conec.commit()

#### Salvando dados dos DataFrames dentro das tabelas do banco de dados;
- Na coluna de sub região, caso o primeiro elemento da quantidade de interesse seja 0, significa que não haviam dados de interesse por estado sucientes para serem analisados, o que implica na não adição desses dados ao banco de dados. Nas outras tabelas isso é feito de forma automática;

In [426]:
nomes_tabelas = ['aumento_assunto', 'sub_regiao', 'interesse_por_data', 'assunto_relacionado']
dataframes = [df_aumentoAssunto, df_subReg, df_dataInteresse, df_assuntoRelac]

#===============Aumento do interesse por assunto relacionado===============#
for index, linha in dataframes[0].iterrows():
    consulta = f"INSERT INTO {nomes_tabelas[0]} (Data_Inicio, Data_Fim, Termo_Busca, Assunto, Porcentagem_Aumento) VALUES (%s, %s, %s, %s, %s)"
    valores = (linha['Data_Inicio'], linha['Data_Fim'], linha['Termo_Busca'], linha['Assunto'], linha['Porcentagem_Aumento'])
    cursor.execute(consulta, valores)
conec.commit()
print(f'dataframe "{nomes_tabelas[0]}" salvo com sucesso')

#===============Sub região===============#
for index, linha in dataframes[1].iterrows():
    consulta = f"INSERT INTO {nomes_tabelas[1]} (Data_Inicio, Data_Fim, Termo_Busca, UF, Quant_Interesse) VALUES (%s, %s, %s, %s, %s)"
    valores = (linha['Data_Inicio'], linha['Data_Fim'], linha['Termo_Busca'], linha['UF'], linha['Quant_Interesse'])
    cursor.execute(consulta, valores)
conec.commit()
print(f'dataframe "{nomes_tabelas[1]}" salvo com sucesso')


#===============Assunto relacionado===============#
for index, linha in dataframes[3].iterrows():
    consulta = f"INSERT INTO {nomes_tabelas[3]} (Data_Inicio, Data_Fim, Termo_Busca, Assunto_Relacionado, Quant_Interesse) VALUES (%s, %s, %s, %s, %s)"
    valores = (linha['Data_Inicio'], linha['Data_Fim'], linha['Termo_Busca'], linha['Assunto_Relacionado'], linha['Quant_Interesse'])
    cursor.execute(consulta, valores)
conec.commit()
print(f'dataframe "{nomes_tabelas[3]}" salvo com sucesso')


#===============Interesse por data===============#
for index, linha in dataframes[2].iterrows():
    consulta = f"INSERT INTO {nomes_tabelas[2]} (Intervalo_De, Intervalo_Ate, Data_Registro, Termo_Busca, Quant_Interesse) VALUES (%s, %s, %s, %s, %s)"
    valores = (linha['Intervalo_De'], linha['Intervalo_Ate'], linha['Data_Registro'], linha['Termo_Busca'], linha['Quant_Interesse'])
    cursor.execute(consulta, valores)
conec.commit()
print(f'dataframe "{nomes_tabelas[2]}" salvo com sucesso')

dataframe "aumento_assunto" salvo com sucesso
dataframe "sub_regiao" salvo com sucesso
dataframe "assunto_relacionado" salvo com sucesso
dataframe "interesse_por_data" salvo com sucesso


#### Finalizando programa e apagando os arquivos csv baixados
- Vale ressaltar que o código está sujeito a erros referentes a alterações no html da página do Google Trends após alguma aualização, sendo necessário uma manutenção para atualizar os xpath's;
- Os parâmetros de tratamento e carga dos dados podem ser redefinidos facilmente para qualquer que seja a solicitação, bastando apenas uma manutenção simples (os dados podem ser interpretados e carregados em dicionários, tabelas únicas, comparações diferentes, etc)

In [427]:
#===Excluindo arquivos originais para permitir que o código seja executado novamente
arquivos_excluir = ['geoMap.csv', 'multiTimeline.csv', 'relatedEntities.csv']

for arquivo in arquivos_excluir:  
    caminho_do_arquivo = f'{endereco}\\\{arquivo}'
    if os.path.exists(caminho_do_arquivo):
        os.remove(caminho_do_arquivo)
        print(f"O arquivo {caminho_do_arquivo} foi excluído com sucesso.")
    else:
        print(f"O arquivo {caminho_do_arquivo} não existe.")

O arquivo C:\Users\josen\trends_api\\geoMap.csv foi excluído com sucesso.
O arquivo C:\Users\josen\trends_api\\multiTimeline.csv foi excluído com sucesso.
O arquivo C:\Users\josen\trends_api\\relatedEntities.csv foi excluído com sucesso.
